In [ ]:
!pip3 install selenium

In [115]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import re
import time
import pandas as pd

In [134]:
# Term u want to search for. eg 'breast'
term = "breast"
# No of Papers u want to search 1,10,100 .
pages = "5"
# Keywords u want to check for eg if u want to find ["Breast cancer","Breast size","Epidemiology","Risk reduction"]
# then write keywords seperated by comma Breast cancer, Breast size, Epidemiology, Risk reduction
keywords = "Breast cancer, Breast size, Epidemiology, Risk reduction"
# Name of output file
output = "Pubmed"
# Path To chrome drivers
path = r'/bin/chromedriver'

In [135]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    cleantext = str.lower(cleantext)
    return cleantext

In [136]:
keywords = keywords.split(",")
keywords = list(map(str.lower,keywords))
keywords = list(map(str.strip,keywords))
print("Keywords : ",keywords)
keywords_dict = {"pmid":[]}
for i in keywords:
    keywords_dict[i] = []
print("Keywords Dictonary : ",keywords_dict)
pages = int(pages)

driver = webdriver.Chrome(path)
print("\nStarting Web Scrapping")
print("Term Searching for : ",term)
driver.get("https://pubmed.ncbi.nlm.nih.gov/?term="+term)
driver.get("https://pubmed.ncbi.nlm.nih.gov/?term="+term)
element = driver.find_element(By.XPATH,"//span[@class='value']")
results = element.get_attribute('innerHTML').split(",")
results = "".join(results)
results = int(results)
print("Total No of results : ",results)
print("No of Records asked to Scan : ",pages)
if pages <= results:
    print("Procedding")
else:
    pages = results
    print("No of Records asked to scan is more than present hence scanning only : ",results)
element = driver.find_element(By.XPATH,"//a[@class='docsum-title']")
action = ActionChains(driver)
action.click(on_element = element)
action.perform()
# driver.get(element.get_attribute('href'))
for i in range(pages):
    print("Index : ",(i+1))
    element = driver.find_element(By.XPATH,"//span[@class='identifier pubmed']")
    print(element.find_element(By.TAG_NAME,"strong").get_attribute('innerHTML'))
    keywords_dict["pmid"].append(element.find_element(By.TAG_NAME,"strong").get_attribute('innerHTML'))
    element = driver.find_element(By.XPATH,"//div[@id='abstract']")
    txt = cleanhtml(element.get_attribute('innerHTML'))
    for j in keywords:
        if txt.find(j) !=-1:
            keywords_dict[j].append(1)
        else:
            keywords_dict[j].append(0)
    time.sleep(2)
    element = driver.find_element(By.XPATH,"//div[@class='next side-link visible']")
    action = ActionChains(driver)
    action.click(on_element = element)
    action.perform()
driver.quit()
data = pd.DataFrame.from_dict(keywords_dict)
print("\n\nData Frame\n\n")
print(data.head())
print("\nWritting data to file ",output,".csv")
data.to_csv(output+".csv")
print("\n\n!!DONE!!")

Keywords :  ['breast cancer', 'breast size', 'epidemiology', 'risk reduction']
Keywords Dictonary :  {'pmid': [], 'breast cancer': [], 'breast size': [], 'epidemiology': [], 'risk reduction': []}


/tmp/ipykernel_69785/2630301366.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)



Starting Web Scrapping
Term Searching for :  breast
Total No of results :  598371
No of Records asked to Scan :  5
Procedding
Index :  1
32027563
Index :  2
24636325
Index :  3
25456291
Index :  4
29098159
Index :  5
30503250


Data Frame


       pmid  breast cancer  breast size  epidemiology  risk reduction
0  32027563              0            0             0               0
1  24636325              0            0             0               0
2  25456291              1            1             1               1
3  29098159              1            0             0               0
4  30503250              0            0             0               0

Writting data to file  Pubmed .csv


!!DONE!!
